## Startup code

In [ ]:
%load_ext autotime
import warnings
warnings.filterwarnings('always')
warnings.simplefilter('always')
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=DeprecationWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")
import glob, os, sys
# import matplotlib.pylab as plt
# from matplotlib import pyplot
import numpy as np
from sklearn.model_selection import StratifiedKFold
import tensorflow as tf
print(tf.__version__)
import pandas as pd
pd.options.mode.chained_assignment = None
import csv
import xml.etree.ElementTree as ET
import pickle
import math
# import matplotlib.pyplot as plt
%config Completer.use_jedi = False
from bs4 import BeautifulSoup
import nltk
import shutil

import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
import gensim
import pprint

## Helper methods

In [ ]:
def save_df(df, file_name, append=False):
    if append:
        df.to_csv(file_name, index=False, quotechar='"', quoting=csv.QUOTE_NONNUMERIC, mode="a", header=False)
    else:
        df.to_csv(file_name, index=False, quotechar='"', quoting=csv.QUOTE_NONNUMERIC)


def load_obj(base_dir, name):
    file_path = os.path.join(base_dir, "pickle", name + ".pkl")
    with open(file_path, 'rb') as f:
        return pickle.load(f)

def save_obj(obj, base_dir, name):
    dir_path = os.path.join(base_dir, "pickle")
    create_director(dir_path)
    file_path = os.path.join(dir_path, name + ".pkl")
    with open(file_path, 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)
    print("Saved object to a file: %s" % (str(file_path)))

def save_df(df, base_dir, file_name):
    file_name = os.path.join(base_dir, file_name)
    df.to_csv(file_name, index=False, quotechar='"', quoting=csv.QUOTE_NONNUMERIC)

def remove_directory(path):
    if os.path.exists(path):
        print("%s path exists and removing it." % path)
        shutil.rmtree(path)

def remove_file(file_name):
    if (os.path.isfile(file_name)):
        print("Output file %s exists and removing it." % file_name)
        os.remove(file_name)

def create_director(dir):
    if(not os.path.exists(dir)):
        print("Creating directory %s." % dir)
        os.makedirs(dir)
    else:
        print("Directory %s already exists and so returning." % dir)


time: 1.91 ms


## Tasks

### Data exploration

#### Pandas

In [ ]:
file_name = "/work/disa_lab/Alamin/SOTorrent/SO/Posts.csv"
file_name = "/work/disa_lab/Alamin/SOTorrent/serverfault/Posts.csv"
# df = pd.read_csv(file_name)
# print(len(df))

time: 307 µs (started: 2021-10-25 10:48:49 -06:00)


In [ ]:
df2 = pd.read_csv(file_name, usecols=["Id", "PostTypeId"])
print(len(df2))

45444575
time: 9min 9s (started: 2021-10-25 10:48:54 -06:00)


In [ ]:
print(df2.info(memory_usage="deep"))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45444575 entries, 0 to 45444574
Data columns (total 2 columns):
 #   Column      Dtype
---  ------      -----
 0   Id          int64
 1   PostTypeId  int64
dtypes: int64(2)
memory usage: 693.4 MB
None
time: 4.32 ms (started: 2021-10-25 10:58:28 -06:00)


In [ ]:
print(df.info(memory_usage="deep"))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 791818 entries, 0 to 791817
Data columns (total 23 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Id                     791818 non-null  int64  
 1   PostTypeId             791818 non-null  int64  
 2   AcceptedAnswerId       145148 non-null  float64
 3   ParentId               485014 non-null  float64
 4   CreationDate           791818 non-null  object 
 5   DeletionDate           0 non-null       float64
 6   Score                  791818 non-null  int64  
 7   ViewCount              302476 non-null  float64
 8   Body                   791106 non-null  object 
 9   OwnerUserId            770595 non-null  float64
 10  OwnerDisplayName       31859 non-null   object 
 11  LastEditorUserId       233461 non-null  float64
 12  LastEditorDisplayName  2544 non-null    object 
 13  LastEditDate           235826 non-null  object 
 14  LastActivityDate       791818 non-nu

In [ ]:
post = df[df['Id'] == 449564].iloc[0]
print(post['Body'])

<p>Say I want all requests to example.com to redirect to abc.com.</p>

<p>How can I make everything redirect to the root of abc.com and not include any paths or parameters included in example.com?</p>

<p>For example</p>

<pre><code>example.com/index.py?var=1 should go to abc.com
</code></pre>

<p>I tried this in my .htaccess</p>

<pre><code>RedirectPermanent / http://www.abc.com
</code></pre>

<p>But it does this:</p>

<pre><code>example.com/index.py?var=1 goes to abc.com/index.py?var=1
</code></pre>

time: 2.95 ms (started: 2021-10-25 09:44:47 -06:00)


#### Dask

In [ ]:

from dask.distributed import Client
client = Client(n_workers=4)

time: 4.18 s (started: 2021-10-25 12:53:59 -06:00)


In [ ]:
print(client.status)

running
time: 621 µs (started: 2021-10-25 12:54:03 -06:00)


In [ ]:
import dask.dataframe as dd

time: 1.32 s (started: 2021-10-25 12:54:04 -06:00)


In [ ]:
file_name = "/work/disa_lab/Alamin/SOTorrent/serverfault/Posts.csv"
df  = dd.read_csv(file_name,  dtype={"Id": "Int64"}, quoting=csv.QUOTE_NONE, encoding='utf-8', blocksize=None, error_bad_lines=False)
# print(df.head)
# print(len(df))

time: 12.3 ms (started: 2021-10-25 12:57:23 -06:00)


In [ ]:
print(len(df))

/home/mdabdullahal.alamin/.conda/envs/latest/lib/python3.7/site-packages/dask/dataframe/io/csv.py:134: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  path_info,
b'Skipping line 6966: expected 23 fields, saw 27\nSkipping line 6979: expected 23 fields, saw 27\nSkipping line 28910: expected 23 fields, saw 35\n'
b'Skipping line 48286: expected 23 fields, saw 25\n'
b'Skipping line 104634: expected 23 fields, saw 24\n'
b'Skipping line 164024: expected 23 fields, saw 28\n'
b'Skipping line 179453: expected 23 fields, saw 26\nSkipping line 179454: expected 23 fields, saw 26\nSkipping line 179458: expected 23 fields, saw 26\nSkipping line 179459: expected 23 fields, saw 26\nSkipping line 179462: expected 23 fields, saw 26\nSkipping line 179463: expected 23 fields, saw 26\nSkipping line 179467: expected 23 fields, saw 24\nSkipping line 179468: expected 23 fields, saw 26\nSkipping line 179469: expected 23 fields, saw 26\nSkipping line 1

KeyboardInterrupt: ignored

time: 34.5 s (started: 2021-10-25 12:57:26 -06:00)


In [ ]:
client.shutdown()

time: 210 ms (started: 2021-10-25 09:42:14 -06:00)


### Convert XML files to CSV files

In [ ]:
import lxml.etree  as ET


def save_df(df, file_name, append=False):
    if append:
        df.to_csv(file_name, index=False, quotechar='"', quoting=csv.QUOTE_NONNUMERIC, mode="a", header=False)
    else:
        df.to_csv(file_name, index=False, quotechar='"', quoting=csv.QUOTE_NONNUMERIC)

def convert_xml_to_csv(xml_file, output_csv_file = None):
    if output_csv_file is None:
        output_csv_file = xml_file[:-4] + ".csv"
    print(output_csv_file)
    df = pd.read_xml(xml_file)
    print("Org Datafrme size: %d" % len(df))
    df.drop_duplicates('Id',inplace=True)
    print("After removing duplicates Datafrme size: %d" % len(df))
    save_df(df, output_csv_file)
    print("Output file has been generated to %s" % output_csv_file)


def append_to_file(rows, output_csv_file, append):
    df = pd.DataFrame(rows)
    df.drop_duplicates('Id', inplace=True)
    save_df(df, output_csv_file, append=append)
    print("%d rows has been appended to the CSV file %s" % (len(df), output_csv_file))
    return len(df)

def convert_xml_to_csv_iteratively(XML_file, columns, output_csv_file = None, threshold = 10000000):
    if output_csv_file is None:
        output_csv_file = XML_file[:-4] + ".csv"

    print("Going to Convert XML files to CSV file")
    context = ET.iterparse(XML_file, recover = True, huge_tree = True, events=("end",))
    # huge_tree = True

    # Variables to process the output
    total_questions = 0
    unique_rows = 0
    rows = []
    cur_count = 0    
    append = False
    _, root = next(context)
    for event, elem in context:
        if elem.tag == "row":
            dic = {}
            for col in COLS:
                dic[col] = elem.attrib.get(col, '')
            rows.append(dic)
             # progress
            if total_questions % 1000000 == 0:
                print('Total Questions: %d' % total_questions)
            elem.clear()
            root.clear()
            total_questions += 1
            cur_count += 1
        if cur_count > threshold:
            unique_rows +=  append_to_file(rows, output_csv_file, append)
            append = True
            cur_count = 0
            rows = []
    if(len(rows) > 0):
        unique_rows += append_to_file(rows, output_csv_file, append)
    print("Total number of rows: %d vs unique_rows: %d" % (total_questions, unique_rows))
    return total_questions

# df.to_csv('my_csv.csv', mode='a', header=False)

time: 2.17 ms (started: 2021-10-18 21:30:14 -06:00)


In [ ]:

file_name = r"E:\\Research\\stackexchange\\study\\raf\\Posts.xml"
file_name = r"/work/disa_lab/Alamin/SOTorrent/test/Posts.xml"
# file_name = r"E:\\Research\\stackexchange\\study\\unix\\Posts.xml"
file_name = os.path.normpath(file_name)
# # file_name = os.path.join(file_name, "Posts.xml")
# convert_xml_to_csv(file_name)
COLS = ["Id", "PostTypeId", "AcceptedAnswerId", "ParentId", "CreationDate", "DeletionDate", "Score", "ViewCount", "Body",
        "OwnerUserId", "OwnerDisplayName", "LastEditorUserId", "LastEditorDisplayName", "LastEditDate", "LastActivityDate",
        "Title", "Tags", "AnswerCount", "CommentCount", "FavoriteCount", "ClosedDate", "CommunityOwnedDate", "ContentLicense"]
convert_xml_to_csv_iteratively(file_name, columns = COLS, threshold=10000000)

Going to Convert XML files to CSV file
Total Questions: 0
59 rows has been appended to the CSV file /work/disa_lab/Alamin/SOTorrent/test/Posts.csv
Total number of rows: 59 vs unique_rows: 59


59

time: 14.1 ms (started: 2021-10-18 21:30:18 -06:00)


In [ ]:
print(df.sample())

### Preprocess Posts

In [ ]:
file_name = "/work/disa_lab/Alamin/SOTorrent/askubuntu/Posts.csv"
df = pd.read_csv(file_name)
print(len(df))

855014
time: 14.8 s (started: 2021-10-18 14:24:48 -06:00)


/home/mdabdullahal.alamin/.conda/envs/latest/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (12) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
body = df['Body'].tolist()

time: 30.5 ms (started: 2021-10-18 14:26:21 -06:00)


In [ ]:

def get_code_segment(html, min_code_length = 10):
    # print(html)
    parsed_html = BeautifulSoup(html, 'html.parser')
    result = ""
    for code in parsed_html.find_all("code"):
        result += code.get_text()        
    if len(result) < min_code_length:
        return np.nan
    return result


def extract_code_segment(input_file, chunk_size=10000000):
    output_csv_file = input_file[:-4] + "_code.csv"
    ids = []
    codes = []


    if (os.path.isfile(output_csv_file)):
        print("Output file %s exists and removing it." % output_csv_file)
        os.remove(output_csv_file)

    append = False
    with pd.read_csv(input_file, usecols=['Id', 'Body'], dtype={'Id': str, 'Body': str}, chunksize=chunk_size) as reader:
        for chunk in reader:
            print(type(chunk))
            print(len(chunk))
            # df = chunk[['Id', 'Body']]
            df = chunk
            df['Body'] = df['Body'].astype(str)            
            df['code'] = df.Body.apply(get_code_segment)
            df = df.drop('Body', axis = 1)
            df.dropna(subset = ["code"], inplace=True)
            save_df(df, output_csv_file, append)
            append = True          

            # break

file_name = "/work/disa_lab/Alamin/SOTorrent/unix/Posts.csv"
extract_code_segment(file_name)



<class 'pandas.core.frame.DataFrame'>
507410
time: 4min 2s (started: 2021-10-18 16:06:15 -06:00)


<class 'pandas.core.frame.DataFrame'>
1000
time: 20 ms (started: 2021-10-18 14:53:39 -06:00)


### TDIDF

In [ ]:
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
import gensim

time: 346 µs (started: 2021-10-25 09:50:39 -06:00)


#### Train

Variables that are stored after processing so that it can be used later.

In [ ]:


# Dictionary = load_obj(BASE_DIR, "Dictionary")
# Corpus = load_obj(BASE_DIR, "Corpus")
# Dictionary = load_obj(BASE_DIR, "Dictionary")
# Corpus = load_obj(BASE_DIR, "Corpus")

time: 7.43 s (started: 2021-10-21 12:51:59 -06:00)


Global important variables

In [ ]:
BASE_DIR = "/work/disa_lab/Alamin/SOTorrent/serverfault/"

time: 326 µs (started: 2021-10-25 11:21:17 -06:00)


In [ ]:
def get_all_codes(BASE_DIR, file_name = "Posts_code.csv"):
    code_segment_file = os.path.join(BASE_DIR, file_name)
    df = pd.read_csv(code_segment_file)
    print("Len of code segment: %d" % len(df))
    ALL_CODE_SEGMENTS = df['code'].tolist()
    ALL_IDs = df['Id'].tolist()    
    print(len(ALL_CODE_SEGMENTS))
    return ALL_CODE_SEGMENTS, ALL_IDs

ALL_CODE_SEGMENTS, ALL_IDs = get_all_codes(BASE_DIR)
save_obj(ALL_IDs, BASE_DIR, "ALL_IDs")

In [ ]:
def get_similarity_metrics(Codes, BASE_DIR):
    Gen_docs = [ word_tokenize(text.lower()) for text in Codes] # tokenize
    Dictionary = gensim.corpora.Dictionary(Gen_docs) # word => Id
    print("Dictionary has been created %d" % len(Dictionary))
    # print(Dictionary.token2id) # word id and its frequency in each document
    Corpus = [Dictionary.doc2bow(gen_doc) for gen_doc in Gen_docs] # word id and its frequency in each document
    print("Corpus has been created")
    # print(word_tokenize(err1))
    TDIDF = gensim.models.TfidfModel(Corpus)
    print("TDIDF has been created") 

    sim_path = os.path.join(BASE_DIR, "TDIDF/")
    remove_directory(sim_path)
    create_director(sim_path)
    Sims = gensim.similarities.Similarity(sim_path, TDIDF[Corpus],
                                        num_features=len(Dictionary))
    print("Sims has been created")
    return Dictionary, Corpus, TDIDF, Sims

Dictionary, Corpus, TDIDF, Sims = get_similarity_metrics(ALL_CODE_SEGMENTS, BASE_DIR)

save_obj(Dictionary, BASE_DIR, "Dictionary")
save_obj(Corpus, BASE_DIR, "Corpus")
save_obj(TDIDF, BASE_DIR, "TDIDF")
save_obj(Sims, BASE_DIR, "Sims")

Directory /work/disa_lab/Alamin/SOTorrent/serverfault/pickle already exists.
Saved object to a file: /work/disa_lab/Alamin/SOTorrent/serverfault/pickle/Dictionary.pkl
Directory /work/disa_lab/Alamin/SOTorrent/serverfault/pickle already exists.
Saved object to a file: /work/disa_lab/Alamin/SOTorrent/serverfault/pickle/Corpus.pkl
Directory /work/disa_lab/Alamin/SOTorrent/serverfault/pickle already exists.
Saved object to a file: /work/disa_lab/Alamin/SOTorrent/serverfault/pickle/TDIDF.pkl
Directory /work/disa_lab/Alamin/SOTorrent/serverfault/pickle already exists.
Saved object to a file: /work/disa_lab/Alamin/SOTorrent/serverfault/pickle/Sims.pkl
time: 6min 45s (started: 2021-10-21 12:03:04 -06:00)


In [ ]:
df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 791818 entries, 0 to 791817
Data columns (total 23 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Id                     791818 non-null  int64  
 1   PostTypeId             791818 non-null  int64  
 2   AcceptedAnswerId       145148 non-null  float64
 3   ParentId               485014 non-null  float64
 4   CreationDate           791818 non-null  object 
 5   DeletionDate           0 non-null       float64
 6   Score                  791818 non-null  int64  
 7   ViewCount              302476 non-null  float64
 8   Body                   791106 non-null  object 
 9   OwnerUserId            770595 non-null  float64
 10  OwnerDisplayName       31859 non-null   object 
 11  LastEditorUserId       233461 non-null  float64
 12  LastEditorDisplayName  2544 non-null    object 
 13  LastEditDate           235826 non-null  object 
 14  LastActivityDate       791818 non-nu

In [ ]:
print("Input text is: %s" % input_text)

#### Small demonstration

In [ ]:
import nltk
nltk.download('punkt')
import pprint as pp

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
code_segments = ["God is Great! I won a lottery", 
                 "Today is such a beautiful day", 
                 "Oh God when this pandemic will be over, I can not take it anymore"]

Tokenize: Split sentences into words

In [ ]:
Gen_docs = [ word_tokenize(text.lower()) for text in code_segments]
for sen in Gen_docs:
    print(sen)

['god', 'is', 'great', '!', 'i', 'won', 'a', 'lottery']
['today', 'is', 'such', 'a', 'beautiful', 'day']
['oh', 'god', 'when', 'this', 'pandemic', 'will', 'be', 'over', ',', 'i', 'can', 'not', 'take', 'it', 'anymore']


Create a Dictionary. Basically assigning each unique word to a unique number

In [ ]:
Dictionary = gensim.corpora.Dictionary(Gen_docs)
print(len(Dictionary))
# pp.pprint(Dictionary.token2id)

25


Create a Bag of words

In [ ]:
# word id and its frequency in each document
Corpus = [Dictionary.doc2bow(gen_doc) for gen_doc in Gen_docs] 
print(Corpus[0])
print(Corpus[1])
print(Corpus[2])

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)]
[(1, 1), (5, 1), (8, 1), (9, 1), (10, 1), (11, 1)]
[(2, 1), (4, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1)]


Term Frequency – Inverse Document Frequency(TF-IDF) is also a bag-of-words model.

In [ ]:
# TFIDF down weights tokens (words) that appears frequently across documents.
TDIDF = gensim.models.TfidfModel(Corpus)
for doc in TDIDF[Corpus]:
    pp.pprint([[Dictionary[id], np.around(freq, decimals=2)] for id, freq in doc])
    break

[['!', 0.47],
 ['a', 0.17],
 ['god', 0.17],
 ['great', 0.47],
 ['i', 0.17],
 ['is', 0.17],
 ['lottery', 0.47],
 ['won', 0.47]]


Gensim Similarity creates an index of all the documents

In [ ]:
sim_path = "/tmp"
Sims = gensim.similarities.Similarity(sim_path, TDIDF[Corpus],
                                        num_features=len(Dictionary))

Make a prediction

In [ ]:
input_text = "Thanks God, for this beautiful day"

tokens = word_tokenize(input_text.lower())
# print(tokens)
query_bow = Dictionary.doc2bow(tokens)
# print(query_bow)
# perform a similarity query against the corpus
query_tf_idf = TDIDF[query_bow]
res = Sims[query_tf_idf] * 100

print("Input text is: %s\n" % input_text)
for i in range(len(res)):    
    print("Input text matches %.2f%% with document:==> %s" % (res[i], code_segments[i]))


Input text is: Thanks God, for this beautiful day

Input text matches 3.14% with document:==> God is Great! I won a lottery
Input text matches 47.58% with document:==> Today is such a beautiful day
Input text matches 28.83% with document:==> Oh God when this pandemic will be over, I can not take it anymore


#### Metadata


In [ ]:
def extract_meta_data(BASE_DIR, file="Posts.csv", columns={}, dtypes={}):
    file_name = os.path.join(BASE_DIR, file)
    df = pd.read_csv(file_name, usecols=columns, dtype=dtypes)
    print("dataframe has been loaded: %d" % len(df))
    print(df.info(memory_usage="deep"))
    save_obj(df, BASE_DIR, "DF_metadata")
    print("Metadata has been stored")

COLS = ['Id', 'PostTypeId', "AcceptedAnswerId", "ParentId", "CreationDate", "ViewCount", "Score"]
Types = {'Id': 'Int64', 'PostTypeId': 'Int64', "AcceptedAnswerId": 'Int64', "ParentId":'Int64', "CreationDate": "str", "ViewCount": 'Int64', "Score": 'Int64'}
# extract_meta_data(BASE_DIR, columns=COLS, dtypes = Types)
extract_meta_data(BASE_DIR="/work/disa_lab/Alamin/SOTorrent/SO/", columns=COLS, dtypes = Types)


### Test Webapplication

In [ ]:
# # BASE_DIR = "/work/disa_lab/Alamin/SOTorrent/unix/"
# BASE_DIR = "/work/disa_lab/Alamin/SOTorrent/serverfault/"
# BASE_DIR = "/work/disa_lab/Alamin/SOTorrent/SO/"
BASE_DIR = "/home/mdabdullahal.alamin/alamin/courses/619_data/DATA/SO/"

sim_path = os.path.join(BASE_DIR, "TDIDF/")
sim_file_path = os.path.join(BASE_DIR, "sim_file.pkl")
print(sim_path, sim_file_path)

/home/mdabdullahal.alamin/alamin/courses/619_data/DATA/SO/TDIDF/ /home/mdabdullahal.alamin/alamin/courses/619_data/DATA/SO/sim_file.pkl
time: 842 µs


In [ ]:
# Sims = gensim.similarities.Similarity.load(sim_file_path)

time: 46.5 ms


In [ ]:
def load_related_variables(root_dir):
    ALL_IDs = load_obj(root_dir, "ALL_IDs")
    print("loaded ALL_IDs and size %d " % len(ALL_IDs))
    Dictionary = load_obj(root_dir, "Dictionary") # Must
    print("loaded Dictionary and size %d " %  len(Dictionary))
    Corpus = load_obj(root_dir, "Corpus")
    print("loaded Corpus and size %d " % len(Corpus ))
    TDIDF = load_obj(root_dir, "TDIDF") # Must
    print("loaded TDIDF and size %d " % sys.getsizeof(TDIDF))
    # Sims = load_obj(root_dir, "Sims") # Must
    Sims = gensim.similarities.Similarity.load(sim_file_path)
    
    print("loaded Sims and size %d " % len(Sims))
    DF_metadata = load_obj(root_dir, "DF_metadata") # Must
    print("loaded DF_metadata and size %d " % len(DF_metadata))
    return ALL_IDs, Dictionary, Corpus, TDIDF, Sims, DF_metadata


ALL_IDs, Dictionary, Corpus, TDIDF, Sims, DF_metadata = load_related_variables(BASE_DIR)


loaded ALL_IDs and size 33623758 
loaded Dictionary and size 2023243 
loaded Corpus and size 33623758 
loaded TDIDF and size 48 
loaded Sims and size 33623758 
loaded DF_metadata and size 45444575 
time: 14min 9s


In [ ]:
import textwrap

from summarizer import Summarizer,TransformerSummarizer
GPT2_model = TransformerSummarizer(transformer_type="GPT2",transformer_model_key="gpt2-medium")

class URL:
    def __init__(self, url, similarity_score, title, summary):
        self.url = url
        self.similarity_score = similarity_score
        self.title = title        
        self.body_summary = summary

    def __str__(self):
        return "Similarity Score: %0.1f\nTitle: %s\nlink: %s\nPost Summary:\n%s\n" % (self.similarity_score, self.title, self.url, self.body_summary)

def get_question_meta_information(df, question_id):
    ques = df[df['Id'] == int(question_id)]
    if(len(ques) < 1):
        return None
    return ques.iloc[0]

def get_matched_post_ids(input, N, Dictionary, TDIDF, Sims, PostIds):
    tokens = word_tokenize(input.lower())
    # print(tokens)
    query_bow = Dictionary.doc2bow(tokens)
    # print(query_bow)
    # perform a similarity query against the corpus
    query_tf_idf = TDIDF[query_bow]
    res = Sims[query_tf_idf]
    indx = res.argsort()[-N:][::-1] # biggest matched Id first
    sim_scores = res[indx] * 100

    post_ids = [PostIds[i] for i in indx]
    print(post_ids)
    return (post_ids, sim_scores)

def get_post_ranking(similarity_tuple, metadata):
    def update_ranking(qid, score):
        if qid not in Map_ranking:
            Map_ranking[qid] = 0.0
        Map_ranking[qid] = Map_ranking[qid] + score

    Map_ranking = {}
    # rank_score = N
    post_ids, similarity_scores = similarity_tuple
    for post_id, post_score in zip(post_ids, similarity_scores):
        post = get_question_meta_information(metadata, post_id)
        rank_score = post_score
        # metadata[metadata['Id'] == post_id].iloc[0]
        # print(post.PostTypeId)
        if(post.PostTypeId == 1):
            # print(type(post.AcceptedAnswerId))
            # Ranking heuristics: If the post does not have accepted answer then deduct some score)
            # Also consider the score of the post in the ranking
            if(pd.isna(post.AcceptedAnswerId) ):
                update_ranking(post.Id, rank_score * 0.95) 
            else:                
                update_ranking(post.Id, rank_score)
        else:
            update_ranking(post.ParentId, rank_score * 0.25)
    # Map_ranking = sorted(Map_ranking.items(), key=lambda x: x[1], reverse=True)
    Map_ranking = dict(sorted(Map_ranking.items(), key=lambda item: item[1], reverse=True))
    return Map_ranking

def generate_url(website, type, qid):
    url = "https://%s.com/%s/%d" % (website, type, qid)
    return url

def gerenerate_results(question_map, website, DF_metadata, N_max=5):
    urls = []
    count = 0
    for qid, similarity_score in question_map.items(): 
        post = get_question_meta_information(DF_metadata, qid)
        if post is not None:
            url_link = generate_url(website, 'q', qid)
            url = URL(url_link, similarity_score, post.Title, prettyfy_long_string(get_body_summary(post.Body)))
            urls.append(url)
            count += 1
        if (count >= N_max):
            break
    return urls

def get_cleaned_text(body):
    soup = BeautifulSoup(body,'html.parser')
    paragraphs = soup.find_all('p')
    res = ""
    for paragraph in paragraphs:
        res += " " + paragraph.get_text()
    return res

def get_text_summary(text, ratio=0.25, min_length=10):
    summary = ''.join(GPT2_model(text, ratio=ratio, min_length=min_length))
    return summary

def get_body_summary(post_body):
    post_body_cleaned = get_cleaned_text(post_body)
    # print(post_body_cleaned)
    post_body_summary = get_text_summary(post_body_cleaned)
    return post_body_summary

def prettyfy_long_string(text, width=100):
    return textwrap.fill(text, width)


# This method uses global variables
def query_and_report_similar_posts(query_text, N_max=30, Dictionary=Dictionary, TDIDF=TDIDF, Sims=Sims, ALL_IDs=ALL_IDs):
    similarity_tuple = get_matched_post_ids(query_text, N_max, Dictionary, TDIDF, Sims, ALL_IDs)
    # print(len(list(similarity_tuple)))

    question_ranks = get_post_ranking(similarity_tuple, DF_metadata)
    print(question_ranks)
    urls = gerenerate_results(question_ranks, "stackoverflow", DF_metadata, N_max=3)
    for url in urls:
        print(url)
    return urls

# query_and_report_similar_posts(example_1)

# similarity_tuple = get_matched_post_ids(example_1, N_max, Dictionary, TDIDF, Sims, ALL_IDs)
# print(len(list(similarity_tuple)))

# question_ranks = get_post_ranking(similarity_tuple, DF_metadata)
# print(question_ranks)

# urls = gerenerate_results(question_ranks, "stackoverflow", DF_metadata, N_max=3)
# for url in urls:
#     print(url)




time: 23.5 s


#### Examples

In [ ]:




example_1 = r'''Java was started but returned exit code=13
C:\Program Files (x86)\Java\jre7\bin\javaw.exe
-Xms40m
-Xmx512m
-XX:MaxPermSize=256m
-jar C:\Program Files\Eclipse-SDK-4.2-win32-x86_64\eclipse\\plugins/org.eclipse.equinox.launcher_1.30v20120522-1813.jar
-os win32
-ws win32
-arch x86_64
-showsplash C:\Program Files\Eclipse-SDK-5.2-win32-x86_64\eclipse\\plugins\org.eclipse.platform_\splash.bmp
-launcher C:\Program Files\Eclipse-SDK-6.2-win32-x86_64\eclipse\eclipse.exe
-name Eclipse
--launcher.library C:\Program Files\Eclipse-SDK-4.2-win32-x86_64\eclipse\\plugins/org.eclipse.equinox.launcher.win32.win32.x86_64_1.1.200.v201205221813\eclipse_1503.dll
-startup C:\Program Files\Eclipse-SDK-4.2-win32-x86_64\eclipse\\plugins/org.eclipse.equinox.launcher_1.30v20120522-1813.jar
--launcher.overrideVmargs
-exitdata 1e30_5c
-vm C:\Program Files (x86)\Java\jre7\bin\javaw.exe
-vmargs
-Xms100m
-Xmx512m
-XX:MaxPermSize=256m
-jar C:\Program Files\Eclipse-SDK-5.2-win32-x86_64\eclipse\\plugins/org.eclipse.equinox.launcher_1.30v20120522-1913.jar
'''

example_2 = r'''
ImportError: Traceback (most recent call last):
  File "C:\Python\lib\site-packages\tensorflow\python\pywrap_tensorflow_internal.py", line 14, in swig_import_helper
    return importlib.import_module(mname)
  File "C:\Python\lib\importlib\__init__.py", line 126, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "<frozen importlib._bootstrap>", line 9518, in _gcd_import
  File "<frozen importlib._bootstrap>", line 961, in _find_and_load
  File "<frozen importlib._bootstrap>", line 950, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 648, in _load_unlocked
  File "<frozen importlib._bootstrap>", line 880, in module_from_spec
  File "<frozen importlib._bootstrap_external>", line 922, in create_module
  File "<frozen importlib._bootstrap>", line 205, in _call_with_frames_removed
ImportError: DLL load failed: Eine DLL-Initialisierungsroutine ist fehlgeschlagen.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Python\lib\importlib\__init__.py", line 126, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "C:\Python\lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "C:\Python\lib\site-packages\tensorflow\python3\pywrap_tensorflow_internal.py", line 20, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "C:\Python\lib\site-packages\tensorflow\python\pywrap_tensorflow_internal.py", line 18, in swig_import_helper
    return importlib.import_module('_pywrap_tensorflow_internal')
ModuleNotFoundError: No module named '_pywrap_tensorflow_internal'
'''

example_3 = '''java.net.ConnectException: Connection refused
        at java.net.PlainSocketImpl.socketConnect(Native Method)
        at java.net.PlainSocketImpl.doConnect(PlainSocketImpl.java:351)
        at java.net.PlainSocketImpl.connectToAddress(PlainSocketImpl.java:213)
        at java.net.PlainSocketImpl.connect(PlainSocketImpl.java:200)
        at java.net.SocksSocketImpl.connect(SocksSocketImpl.java:432)
        at java.net.Socket.connect(Socket.java:529)
        at java.net.Socket.connect(Socket.java:478)
        at java.net.Socket.<init>(Socket.java:375)
        at java.net.Socket.<init>(Socket.java:189)
        at TCPClient.main(Sserver.java:13)'''


time: 3.74 ms


In [ ]:
query_and_report_similar_posts(example_1)

[11461607, 5766381, 14304595, 34018392, 35411373, 57623260, 25022892, 35175286, 11177690, 9120694, 30335092, 41768259, 20398964, 55953092, 44509769, 14211053, 28501820, 15178544, 41531847, 2444831, 7412838, 7090232, 48542106, 12537289, 32759090, 31347750, 31745847, 34083129, 52673928, 51284868]
Score: 95.7
Title: Can't start Eclipse - Java was started but returned exit code=13
link: https://stackoverflow.com/q/11461607
Post Summary:
I am trying to get my first taste of Android development using Eclipse. Similar questions seem to
indicate that it's a 32-bit/64-bit conflict, but I'm 99% positive that I downloaded 64-bit versions
of both Eclipse and Java (RE 7u5), which I chose because I have 64-bit Windows 7.

Score: 84.2
Title: Display main class name of each JVM available like VisualVM
link: https://stackoverflow.com/q/5766381
Post Summary:
VisualVM does it really nice, each full main class name is displayed in the left side-bar
navigation. From the current JVM its easy to obtain all t

In [ ]:
query_and_report_similar_posts(example_2)

[52366122, 49784472, 46146535, 45029587, 55741105, 52059663, 44825115, 45935386, 50542476, 49113497, 49912253, 49629901, 44212207, 43751418, 44058327, 43901332, 51012098, 51017220, 46201392, 45693922, 49093362, 44685975, 45265977, 45008373, 46994743, 45638252, 43942185, 45114656, 49468010, 45461927]
Score: 96.8
Title: "import tensorflow" error on windows with python 3.6.2 CPU
link: https://stackoverflow.com/q/49784472
Post Summary:
I just wanted to try out tensorflow but whenever I try to execute the following, I get an error. I
can anyway find the dll with the "where" command: My PC is running on Windows 10.

Score: 94.8
Title: python tensorflow import dll load failed
link: https://stackoverflow.com/q/49113497
Post Summary:
I installed latest python 3.6.4 x64 version  and then installed tensorflow for cpu-only with pip3
however when I tried to import tensorflow in python it showed me the error below I am sure that I
have installed Microsoft Visual C++ 2015 Redistributable(x64) so it w

In [ ]:
query_and_report_similar_posts(example_3)

IndexError: ignored

time: 3.65 s


In [ ]:


# N_max = 30
# N_res = 5

# post_ids, similarities = get_matched_post_ids(example_1, N_max, Dictionary, TDIDF, Sims, ALL_IDs)
# question_ranks = get_post_ranking(post_ids, DF_metadata)
# print(question_ranks)
# # print(question_ranks.keys())

# # urls = renerate_results(question_ranks.keys(), "serverfault", DF_metadata)
# urls = renerate_results(question_ranks.keys(), "stackoverflow", DF_metadata)
# for url in urls:
#     print(url)

#### experiments


In [ ]:
question_ranks = get_post_ranking(post_ids, DF_metadata)
print(question_ranks)
print(question_ranks.keys())

urls = get_urls(question_ranks.keys(), "serverfault")
print(urls)

{538922: 22.0, 962011: 20.0, 449564: 15.0, 446: 14.5, 442821: 14.0, 547123: 13.5, 218078: 13.0, 93182: 12.5, 173958: 12.0, 24130: 11.5, 109067: 10.5, 911754: 9.5, 230271: 9.0, 99433: 8.5, 390144: 8.0, 6128: 7.5, 425296: 7.0, 188956: 6.5, 94014: 6.0, 69114: 6.0, 153876: 5.5, 90481: 5.0, 511115: 4.5, 9646: 4.5, 790114: 4.0, 1008185: 3.5, 959084: 3.0, 48031: 1.5, 146485: 1.0}
dict_keys([538922, 962011, 449564, 446, 442821, 547123, 218078, 93182, 173958, 24130, 109067, 911754, 230271, 99433, 390144, 6128, 425296, 188956, 94014, 69114, 153876, 90481, 511115, 9646, 790114, 1008185, 959084, 48031, 146485])
['https://serverfault.com/q/538922', 'https://serverfault.com/q/962011', 'https://serverfault.com/q/449564', 'https://serverfault.com/q/446', 'https://serverfault.com/q/442821', 'https://serverfault.com/q/547123', 'https://serverfault.com/q/218078', 'https://serverfault.com/q/93182', 'https://serverfault.com/q/173958', 'https://serverfault.com/q/24130', 'https://serverfault.com/q/109067', '

In [ ]:
res = Sims[query_tf_idf]
print(len(res))
print(type(res))


322719
<class 'numpy.ndarray'>
time: 56.3 ms (started: 2021-10-20 18:04:27 -06:00)


In [ ]:
for i in idx:
    print(ALL_CODE_SEGMENTS[i])
    print("*" * 20)
    # break

In [ ]:
print(ALL_CODE_SEGMENTS[7])

#### Examples

In [ ]:
err_test = '''ModuleNotFoundError                       Traceback (most recent call last)
<ipython-input-68-e2bb30455e63> in <module>
     24 import matplotlib.pyplot as plt
     25 get_ipython().run_line_magic('config', 'Completer.use_jedi = False')
---> 26 from bs4 import BeautifulSoup
     27 import nltk

ModuleNotFoundError: No module named 'bs4'
'''

time: 343 µs (started: 2021-10-20 17:27:01 -06:00)


### Text Summarization

In [ ]:
!pip install ipython-autotime
%load_ext autotime

In [ ]:
!pip install bert-extractive-summarizer

In [ ]:
from summarizer import Summarizer,TransformerSummarizer

In [ ]:
body = '''
       Scientists say they have discovered a new species of orangutans on Indonesia’s island of Sumatra.
The population differs in several ways from the two existing orangutan species found in Sumatra and the neighboring island of Borneo.
The orangutans were found inside North Sumatra’s Batang Toru forest, the science publication Current Biology reported.
Researchers named the new species the Tapanuli orangutan. They say the animals are considered a new species because of genetic, skeletal and tooth differences.
Michael Kruetzen is a geneticist with the University of Zurich who has studied the orangutans for several years. He said he was excited to be part of the unusual discovery of a new great ape in the present day. He noted that most great apes are currently considered endangered or severely endangered.
Gorillas, chimpanzees and bonobos also belong to the great ape species.
Orangutan – which means person of the forest in the Indonesian and Malay languages - is the world’s biggest tree-living mammal. The orange-haired animals can move easily among the trees because their arms are longer than their legs. They live more lonely lives than other great apes, spending a lot of time sleeping and eating fruit in the forest.
The new study said fewer than 800 of the newly-described orangutans exist. Their low numbers make the group the most endangered of all the great ape species.
They live within an area covering about 1,000 square kilometers. The population is considered highly vulnerable. That is because the environment which they depend on is greatly threatened by development.
Researchers say if steps are not taken quickly to reduce the current and future threats, the new species could become extinct “within our lifetime.”
Research into the new species began in 2013, when an orangutan protection group in Sumatra found an injured orangutan in an area far away from the other species. The adult male orangutan had been beaten by local villagers and died of his injuries. The complete skull was examined by researchers.
Among the physical differences of the new species are a notably smaller head and frizzier hair. The Tapanuli orangutans also have a different diet and are found only in higher forest areas.
There is no unified international system for recognizing new species. But to be considered, discovery claims at least require publication in a major scientific publication.
Russell Mittermeier is head of the primate specialist group at the International Union for the Conservation of Nature. He called the finding a “remarkable discovery.” He said it puts responsibility on the Indonesian government to help the species survive.
Matthew Nowak is one of the writers of the study. He told the Associated Press that there are three groups of the Tapanuli orangutans that are separated by non-protected land.He said forest land needs to connect the separated groups.
In addition, the writers of the study are recommending that plans for a hydropower center in the area be stopped by the government.
It also recommended that remaining forest in the Sumatran area where the orangutans live be protected.
I’m Bryan Lynn.

        '''

body2 = '''I've got some hobby servos and I'd like to be able to control them (via an Arduino) so they will either go to the angle I set, or put them in a 'free running' mode, where the load will take them wherever it goes.</p>
Is this even possible, or am I just going to end up stripping the gears?
My first thought is to simply kill the power to the servo, but the force required to move them in that state is more than I'd like.
If it is possible, am I looking at a hardware change, or could I do it in software?
Can these be improved, or can their relative pros and cons be altered, and are there other gaits worth considering?'''

body3 = ''' Your ModelForm save() method is throwing away the return value of the superclass save. You need to return it, eg.
As a general rule: If you know for SURE that a method you are overriding never returns anything, then you don't have to worry about returning anything. But if there's any chance that a method you are overriding could return something useful (now or in the future), then play it safe and return whatever they return. It'll save you a lot of headaches later on.

One thing to think about here, is that you have a small race condition, where if two people order within a few milliseconds of each other, it is possible that you could end up thinking that you have enough stock because customer #1 checked the stock.. it was ok.. then customer #2 checks the stock before customer #1's order was saved. This would cause both orders to be saved when there may be only enough stock for the first order.
'''

In [ ]:
bert_model = Summarizer()

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.25G [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [ ]:
bert_summary = ''.join(bert_model(body3, min_length=50))
print(len(bert_summary))

250


In [ ]:
print(bert_summary)


Your ModelForm save() method is throwing away the return value of the superclass save. But if there's any chance that a method you are overriding could return something useful (now or in the future), then play it safe and return whatever they return.


In [ ]:
GPT2_model = TransformerSummarizer(transformer_type="GPT2",transformer_model_key="gpt2-medium")

Downloading:   0%|          | 0.00/718 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

In [ ]:
full = ''.join(GPT2_model(body3, min_length=60))
print(full)

Your ModelForm save() method is throwing away the return value of the superclass save. But if there's any chance that a method you are overriding could return something useful (now or in the future), then play it safe and return whatever they return.


#### Stackexchange dataset

In [ ]:
print(len(ALL_IDs))
print(ALL_IDs[:10])

322719
[1, 12, 60, 69, 104, 107, 157, 203, 277, 284]
time: 1.12 ms


In [ ]:
# file_name = "/work/disa_lab/Alamin/SOTorrent/serverfault/Posts.csv"
# file_name = "/work/disa_lab/Alamin/SOTorrent/SO/Posts_code.csv"
df_all = pd.read_csv(file_name)
print(len(df_all))

33623758
time: 3min 43s


In [ ]:
df_all.dtypes

Id       int64
code    object
dtype: object

time: 6.34 ms


In [ ]:
def get_question_meta_information(df, question_id):
    ques = df[df['Id'] == str(question_id)].iloc[0]
    return ques
question = get_question_meta_information(df_all, 630453)
print(question)

In [ ]:
print(len(df_all))

791818
time: 1.15 ms


### Error generation

#### Error 1

In [ ]:
import dask


ModuleNotFoundError: ignored

In [ ]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

predictions = model(x_train[:1]).numpy()
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])
model.fit(x_train, y_train, epochs=1)

Epoch 1/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2955 - accuracy: 0.9139
Epoch 2/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1423 - accuracy: 0.9575
Epoch 3/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1072 - accuracy: 0.9677
Epoch 4/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0871 - accuracy: 0.9734
Epoch 5/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0759 - accuracy: 0.9761


In [ ]:
model_save_file = "/model_saved_file"
model.save(model_save_file)

INFO:tensorflow:Assets written to: /model_saved_file/assets
